In [0]:
# Atividade 3 - Pocco

In [0]:
# Cmd 3 - Declarar variável de conexão com o Blob Storage

In [0]:
# Blob Storage
storage_account_access_key = ""

In [0]:
# Cmd 5 - Fazer a configuração da conexão do Blob Storage com o Databricks através do Spark

In [0]:
def configurar_conexao_databricks_blob(storage_account_access_key):
    
    spark.conf.set(
      "fs.azure.sas.mateus-nikolofski.stgestudos.blob.core.windows.net",
      f"{storage_account_access_key}")



In [0]:
# Cmd 7 - Carregar o arquivo CSV do Blob Storage

In [0]:
def fazer_load_arquivo_csv():
    
    df_csv = spark.read.format("csv").option('header', True).option('delimiter', ';').load("wasbs://mateus-nikolofski@stgestudos.blob.core.windows.net/Atividade3_Pocco/Bronze/orders.csv")
    return df_csv

In [0]:
# Cmd 9 - Escrever no Blob Storage

In [0]:
def escrever_arquivo_no_blob(df, folder):

    df.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(f"wasbs://mateus-nikolofski@stgestudos.blob.core.windows.net/Atividade3_Pocco/{folder}/Orders_files")

In [0]:
# Cmd 11 - Renomear arquivo no Blob que começa com "part"

In [0]:
def renomear_arquivo_part_no_blob(folder):
    files = dbutils.fs.ls(f"wasbs://mateus-nikolofski@stgestudos.blob.core.windows.net/Atividade3_Pocco/{folder}/Orders_files")
    output_file = [x for x in files if x.name.startswith("part-")]
    dbutils.fs.mv(output_file[0].path, "%s/Orders.csv" % f"wasbs://mateus-nikolofski@stgestudos.blob.core.windows.net/Atividade3_Pocco/{folder}/Orders_files")

In [0]:
# Cmd 13 - Trocar os espaços nos Headers por underline("_"), pois, facilita a utilização das queries SQL pelo Spark

In [0]:
def substituir_espaço_por_underline(df):
    
    j = []
    for i in df.columns:
        t = i
        if " " in i:
            t = i.replace(" ", "_")
            df = df.withColumnRenamed(i, t)
        j.append(t)
    return df


In [0]:
# Cmd 15 - Criar uma view para utilizar na query SQl como tabela

In [0]:
def criar_view(df):
    
    df.createOrReplaceTempView("pocco_stage")
    return "pocco_stage"

In [0]:
# Cmd 17 - Tranformar os tipos dos dados, trocar vírgulas por pontos e alterar a data em um formato padrão SQL

In [0]:
def transformar_dados(pocco_stage):

    new_lista = spark.sql(f"select Region, Country, Item_Type as `Item Type`,\
    Sales_Channel as `Sales Channel`, Order_Priority as `Order Priority`,\
    date_format(to_date(replace(Order_Date, '/','-'), 'dd-MM-yyyy'), 'yyyy-MM-dd') as `Order Date`,\
    cast(Order_ID as int) as `Order ID`,\
    date_format(to_date(replace(Ship_Date, '/','-'), 'dd-MM-yyyy'), 'yyyy-MM-dd') as `Ship Date`,\
    cast(Units_Sold as int) as `Units Sold`, cast(replace(Unit_Price, ',', '.') as float) as `Unit Price`,\
    cast(replace(Unit_Cost, ',', '.') as float) as `Unit Cost`,\
    cast(replace(Total_Revenue, ',', '.') as float) as `Total Revenue`,\
    cast(replace(Total_Cost, ',', '.') as float) as `Total Cost`,\
    cast(replace(Total_Profit,',', '.') as float) as `Total Profit` from {pocco_stage}").collect()
    return new_lista

In [0]:
# Cmd 19 - Utilizar Spark para transformar uma lista em um dataframe

In [0]:
def tranformar_em_dataframe(new_lista):
    
    final_df = spark.createDataFrame(new_lista)
    return final_df

In [0]:
# Cmd 21 - A função main que é responsável pelo orquestração das funções

In [0]:
def main():
    configurar_conexao_databricks_blob(storage_account_access_key)
    df_csv = fazer_load_arquivo_csv()
    df_underline = substituir_espaço_por_underline(df_csv)
    view = criar_view(df_underline)
    lista_transformada = transformar_dados(view)
    df_transformado =  tranformar_em_dataframe(lista_transformada)
    escrever_arquivo_no_blob(df_transformado, "Silver")
    renomear_arquivo_part_no_blob("Silver")

    
   

In [0]:
# Cmd 23 - Chamada da função main()

In [0]:
main()